# A Distributed Matrix Data Structure and Its Statistical Applications on PyTorch

**The Programming Workshop at the Inaugural Kenneth Lange Symposium, Feb 21-22, 2020**

_Seyoon Ko and Joong-Ho (Johann) Won_

## Synopsis

We developed a distributed matrix operation package suitable for distributed matrix-vector operations and distributed tall-and-thin (or wide-and-short) matrices.
The code runs on both multi-node machines and multi-GPU machines using PyTorch.
We apply this package for four statistical applications, namely, nonnegative matrix factorization (NMF), multidimensional scaling (MDS), positron emission tomography (PET), and $\ell_1$-regularized Cox regression.
In particular, $\ell_1$-regularized Cox regression with the UK Biobank dataset was the biggest joint multivariate survival analysis to our knowledge. 
In this workshop, we provide small examples that run on a single node, and demonstrate multi-GPU usage on our own machine.

* Brief introduction to PyTorch operations
* `torch.distributed` package
* Distributed matrix data structure in package `dist_stat`
* Applications: Nonnegative matrix factorization and $\ell_1$-penalized Cox regression
* Demonstration on multi-GPU machine

## Introduction to PyTorch

PyTorch is an optimized tensor library for deep learning using GPUs and CPUs. It has two goals of development:
* A replacement for NumPy to use the power of GPUs $\rightarrow$ optimization of numerical operations
* A deep learning research platform that provides maximum flexibility and speed $\rightarrow$ optimization of automatic gradient computation e.g. backpropagation

We are trying to exploit the former in a distributed environment.

## Basic PyTorch Operations

We introduce simple operations on PyTorch. Note that Python uses 0-based, row-major ordering, like C and C++ (cf. R and Julia have 1-based, column-major ordering). First we import the PyTorch
library. This is similar to `library()` in R and equivalent to `import ...` in Julia.

In [ ]:
import torch

In [ ]:
torch.__version__

### Tensor Creation

One may create an uninitialized tensor. This creates a 3 × 4 tensor (matrix).

In [ ]:
torch.empty(3, 4) # uninitialized tensor. Julia equivalent: Array{Float32}(undef, 3, 4)

The following is equivalent to `set.seed()` in R.

In [ ]:
torch.manual_seed(100) # Julia equivalent: Random.seed!(100)

This generates a tensor initialized with random values from (0, 1).

In [ ]:
y = torch.rand(3, 4) # from Unif(0, 1). 
y

We can also generate a tensor filled with zeros or ones.

In [ ]:
z = torch.ones(3, 4) # torch.zeros(3, 4)
z

A tensor can be created from standard Python data.

In [ ]:
w = torch.tensor([3, 4, 5, 6])
w

A tensor can be created in certain datatype (default: float32) and on certain device (default: CPU) of choice 

In [ ]:
# double precision
w = torch.tensor([3, 4, 5, 6], dtype=torch.float64)
w

In [ ]:
# # on GPU number zero. will not run if CUDA GPU is not present.
# w = torch.tensor([3, 4, 5, 6], device='cuda:0')
# w

Shape of a tensor can be accessed by appending `.shape` to the tensor name.

In [ ]:
z.shape

### Casting

A tensor can change datatype and location by the method `.to()`. The arguments are similar to choosing datatype and device of the new tensor.

In [ ]:
w = w.to(device = "cpu", dtype=torch.int32)
w

### Indexing

The following are standard method of indexing tensors.

In [ ]:
y[2, 3] # indexing: zero-based, returns a 0-dimensional tensor

The indexing always returns a (sub)tensor, even for scalars (treated as zero-dimensional tensors).
A standard Python number can be returned by using .item().

In [ ]:
y[2, 3].item() # A standard Python floating-point number

To get a column from a tensor, we use the indexing as below. The syntax is similar but slightly
different from R.

In [ ]:
y[:, 3] # 3rd column. The leftmost column is 0th. cf. y[, 4] in R

The following is for taking a row.

In [ ]:
y[2, :] # 2nd row. The top row is 0th. cf. y[3, ] in R

### Simple operations

Here we provide an example of simple operations on PyTorch. Addition using the operator ‘+’ acts
just like anyone can expect:

In [ ]:
x = y + z # a simple addition.
x

Here is another form of addition.

In [ ]:
x = torch.add(y, z) # another syntax for addition

The operators ending with an underscore (`_`) changes the value of the tensor in-place. Otherwise, the argument never changes. Unlike methods ending with `!` in Julia, this rule is strictly enforced in PyTorch. (The underscore determines usage of the keyword `const` in C++-level.)

In [ ]:
y.add_(z) # in-place addition

### Concatenation

We can concatenate the tensors using the function `cat()`, which resembles `c()`, `cbind()`, and
`rbind()` in R, `cat()`, `vcat()`, `hcat()` in Julia. The second argument indicates the dimension that the tesors are concatenated
along: zero means by concatenation by rows, and one means by columns.

In [ ]:
torch.cat((y, z), 0) # along the rows

In [ ]:
torch.cat((y, z), 1) # along the columns

### Reshaping

One can reshape a tensor, like changing the attribute `dim` in R.

In [ ]:
y.view(12) # 1-dimensional array

Up to one of the arguments of `view()` can be −1. The size of the reshaped tensor is inferred
from the other dimensions.

In [ ]:
# reshape into (6)-by-2 tensor;
# (6) is inferred from the other dimension
y.view(-1, 2)

### Basic statistics

Calling `.sum()`, `.mean()`, `.std()` methods of a tensor do the obvious. Optional argument determines the dimension of reduction.

In [ ]:
y

In [ ]:
y.sum()

In [ ]:
y.sum(0) # reduces rows, columnwise sum

In [ ]:
y.sum(1) # reduces columns, rowwise sum

In [ ]:
y.sum((0, 1)) # reduces rows and columns -> a single number.

In [ ]:
y.mean()

In [ ]:
y.mean(0)

In [ ]:
y.std(1)

### Linear Algebra

Matrix transpose is performed by appending `.t()` to a tensor. Matrix multiplication is carried out by the method `torch.mm()`.

In [ ]:
torch.mm(y, z.t()) # Note: y is 3 x 4, z is 3 x 4. 

## `torch.distributed`: Distributed subpackage for PyTorch

`torch.distributed` is the subpackage for distributed operations on PyTorch. The interface is mostly inspired by the message passing interface (MPI). The available backends are:

* Gloo, a collective communication library developed by Facebook, included in PyTorch. Full support for CPU, partial collective communication only for GPU.
* MPI, a good-old communication standard. The most flexible, but PyTorch needs to be compiled from its source to use it as a backend. Full support for GPU if the MPI installation is "CUDA-aware".
* NCCL, Nvidia Collective Communications Library, collective communication only for multiple GPUs on the same machine.

For this workshop, we use Gloo for its full functionalities on CPU and runnability on Jupyter Notebook. The experiments in our paper use MPI for running multi-node setting and multi-GPU setting with basically the same code. The interface below is specific for Gloo backend. For MPI backend, please consult with a section from [distributed package tutorial](https://pytorch.org/tutorials/intermediate/dist_tuto.html#communication-backends) or [our example code](https://github.com/kose-y/dist_stat/tree/master/examples).

In [ ]:
import os
import torch
import torch.distributed as dist
from torch.multiprocessing import Process

def init_process(rank, size, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size)

def run_process(size, fn):
    processes = []
    for rank in range(size):
        p = Process(target=init_process, args=(rank, size, fn))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()

### Point-to-point communication

![](https://pytorch.org/tutorials/_images/send_recv.png)

Figure courtesy of: https://pytorch.org/tutorials/_images/send_recv.png

In [ ]:
"""Blocking point-to-point communication."""

def point_to_point(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor += 1
        # Send the tensor to process 1
        dist.send(tensor=tensor, dst=1)
    elif rank == 1:
        # Receive tensor from process 0
        dist.recv(tensor=tensor, src=0)
    dist.barrier()
    print('Rank ', rank, ' has data ', tensor[0])

In [ ]:
run_process(4, point_to_point)

### Collective communication

| | | 
|:---|:---|
| ![](https://pytorch.org/tutorials/_images/scatter.png) | ![](https://pytorch.org/tutorials/_images/gather.png) |
| Scatter | Gather |
| ![](https://pytorch.org/tutorials/_images/reduce.png) | ![](https://pytorch.org/tutorials/_images/all_reduce.png) |
| Reduce | All-reduce |
| ![](https://pytorch.org/tutorials/_images/broadcast.png) | ![](https://pytorch.org/tutorials/_images/all_gather.png) |
| Broadcast | All-gather |

Table courtesy of: https://pytorch.org/tutorials/intermediate/dist_tuto.html


In [ ]:
def broadcast(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor[0] = 7
    dist.broadcast(tensor, src=0)
    print('Rank ', rank, ' has data ', tensor[0])

In [ ]:
run_process(4, broadcast)

In [ ]:
def reduce(rank, size):
    tensor = torch.ones(1)
    dist.reduce(tensor, 3)
    print('Rank ', rank, ' has data ', tensor[0])

In [ ]:
run_process(4, reduce)

In [ ]:
def all_reduce(rank, size):
    tensor = torch.ones(1)
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
    print('Rank ', rank, ' has data ', tensor[0])

In [ ]:
run_process(4, all_reduce)

In [ ]:
def all_gather(rank, size):
    tensors = [torch.zeros(1) for i in range(size)]
    dat = torch.zeros(1)
    dat[0] += rank
    dist.all_gather(tensors, dat)
    print('Rank ', rank, ' has data ', tensors)

In [ ]:
run_process(4, all_gather)

The below is the code for simple Monte Carlo $\pi$ estimation. 100,000 $x_i$ and $y_i$ are sampled from $Unif(0,1)$ for each process, and we measure the proportion of $(x_i, y_i)$s inside the unit quartercircle. 

In [ ]:
def mc_pi(n, size):
    # this code is executed on each process.
    x = torch.rand((n), dtype=torch.float64)
    y = torch.rand((n), dtype=torch.float64)
    # compute local estimate of pi
    r = torch.mean((x**2 + y**2 < 1).to(dtype=torch.float64))*4
    dist.all_reduce(r) # sum of 'r's in each device is stored in 'r'
    return r / size

def run_mc_pi(rank, size):
    n = 100000
    torch.manual_seed(100 + rank)
    r = mc_pi(n, size)
    if rank == 0:
        print(r.item())

In [ ]:
run_process(4, run_mc_pi)

## `distmat`: Distributed Matrices on PyTorch

Using the tensor operations and communication package, we created a data structure for a distributed matrix. 
* Each process (enumerated by rank) holds a contiguous block of the full data matrix by rows or columns.
* The data may be a sparse matrix. 
* If GPUs are involved, each process controls a GPU whose index matches the process rank. 
* From now on: [100] × 100 matrix split over four processes means...
    * Rank 0 process keeps rows [0:25). of the matrix, in row-major ordering.
    * Rank 3 process keeps rows [75:100).
* Limitation: length of distributed dimension should be divisible by number of processes.

In [ ]:
import dist_stat.distmat as distmat

### Creation

- `distgen_ones()`: Creates a distributed matrix filled with ones
- `distgen_zeros()`: Creates a distributed matrix filled with zeros
- `distgen_uniform()`: Creates a distributed matrix from uniform distribution
- `distgen_normal()`: Creates a distributed matrix from stndard normal distribution 

In [ ]:
def create_unif(rank, size):
    A = distmat.distgen_uniform(8, 8, TType=torch.DoubleTensor)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)   

In [ ]:
run_process(4, create_unif)

A distributed matrix can also be created from local chunks.

In [ ]:
def from_chunks(rank, size):
    torch.manual_seed(100 + rank)
    chunk = torch.randn(2, 4)
    print("rank ", rank, "has chunk", chunk)
    A = distmat.THDistMat.from_chunks(chunk)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)    
    if rank == 0:
        print(A.shape)

In [ ]:
run_process(4, from_chunks)

Data can be distributed from the master process.

In [ ]:
def dist_data(rank, size):
    if rank == 0:
        data = torch.rand(4, 2)
        print("master data: ", data)
    else:
        data = None
    
    data_dist = distmat.dist_data(data, src=0, TType=torch.DoubleTensor)
    print('data_dist: ', 'Rank ', rank, ' has data ', data_dist.chunk)
    dist.barrier()
    print(data_dist.shape) # shape of the distributed matrix
    dist.barrier()
    print(data_dist.sizes) # sizes along the distributed dimension

In [ ]:
run_process(4, dist_data)

Remark: The default is to create a row-major matrix. They can easily be changed to a column-major matrix by transposition.

### Elementwise operations

Some of the basic functions work naturally. 

In [ ]:
def elemwise_1(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_uniform(4, 2)
    print("B: rank ", rank, "has chunk", B.chunk)    
    C = A + B
    print("C: rank ", rank, "has chunk", C.chunk)    
    C += A
    print("C: rank ", rank, "has chunk", C.chunk)
    
    logC = C.log()
    print("logC: rank ", rank, "has chunk", logC.chunk) 

In [ ]:
run_process(4, elemwise_1)

Broadcasting (similar to Julia's dot broadcasting) also works as expected:

In [ ]:
def dim_broadcasting(rank, size):
    A = distmat.distgen_uniform(4, 2) # [4] x 2 
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_ones(4, 1) # [4] x 1
    print("B: rank ", rank, "has chunk", B.chunk)
    A += B # B treated as [4] x 2 matrix
    print("A: rank ", rank, "has chunk", A.chunk)
    C = 2 * torch.ones(1, 2, dtype=torch.float64) # 1 x 2
    A += C # C treated as [4] x 2 matrix
    print("A: rank ", rank, "has chunk", A.chunk)     

In [ ]:
run_process(4, dim_broadcasting)

For general functions, we have `.apply()` and `.apply_binary()`.

In [ ]:
def elemwise_2(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_uniform(4, 2)
    print("B: rank ", rank, "has chunk", B.chunk) 
    Asqp1 = A.apply(lambda x: x**2 + 1)
    print("Asqp1: rank ", rank, "has chunk", Asqp1.chunk)    
    AsqpBsq = A.apply_binary(B, lambda x, y: x**2 + y**2)
    print("AsqpBsq: rank ", rank, "has chunk", AsqpBsq.chunk) 

In [ ]:
run_process(4, elemwise_2)

### Reductions (sum, max, min)

Summations, minimums, and maximums can be carried out in a way similar to local tensors.

In [ ]:
def reductions(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    print("sum of A: ", A.sum())
    print("maximum of A: ", A.max())
    print("minimum of A: ", A.min())
    
    sumA_row = A.sum(0) # row sum, a tensor with the same values on all processes 
    sumA_col = A.sum(1) # col sum, a distributed matrix
    print("row sum of A: ", sumA_row)
    
    print("sumA_col: rank ", rank, "has chunk", sumA_col.chunk)

In [ ]:
run_process(4, reductions)

### Diagonals

In [ ]:
def diagonals(rank, size):
    if rank == 0:
        p = 4
        data = torch.randn(p, p)
        print("master data: ", data)
    else:
        data = None
        
    data_dist = distmat.dist_data(data, src=0, TType=torch.DoubleTensor)
    
    diag1 = data_dist.diag() # distributed diagonal
    print("diag1: rank ", rank, "has chunk", diag1.chunk)
    
    diag2 = data_dist.diag(distribute=False) # diagonal gathered in each process
    print("diag2: ", diag2)
    
    data_dist.fill_diag_(0) # fill the diagonals with zeros
    print("data_dist: rank ", rank, "has chunk", data_dist.chunk)

In [ ]:
run_process(4, diagonals)

### Matrix multiplications

Six different scenarios of matrix-matrix multiplications, each representing a different configuration of the split dimension of two input
matrices and the output matrix, were considered and implemented. 

| Secnario | $A$ | $B$ | $AB$ | Description | Usage |
|:---|:---|:---|:---|:---|:---|
| 1 | $r \times$ [$p$] | $[p] \times q$ | $r \times$ [$q$]| Inner product, result distributed. | $V^T X$ |
| 2 | $[p] \times q$ | $[q] \times r$ | $[p] \times r$ | Fat matrix multiplied by a thin and tall matrix. | $X W^T$ |
| 3 | $r \times$ [$p$] | $[p] \times s$ | $r \times s$   | Inner product, result broadcasted. Inner product between two thin matrices. | $V^T V$, $W W^T$ |                                                                           
| 4 | $[p] \times r$ | $r \times$ [$q$] | $[p] \times q$ | Outer product, may require large amount of memory. For computing objective function. | $VW$ |
| 5 | $[p] \times r$ | $r \times s$   | $[p] \times s$ | A distributed matrix multiplied by a small, distributed matrix. | $VC$ where $C = WW^T$; $CW$ where $C = V^T V$ (transposed) |
| 6 | $r \times$ [$p$] | $p \times s$   | $r \times s$   | A distributed matrix multiplied by a thin-and-tall broadcasted matrix. | Matrix-broadcasted vector multiplications. |



The implementation of each case is carried
out using the collective communication directives. Matrix multiplication scenarios are automatically selected based on the shapes of the input matrices A and
B, except for the Scenarios 1 and 3 sharing the same input structure. Those two are further
distinguished by the shape of output, AB. The nonnegative matrix factorization involves Scenarios 1 to 5.
Scenario 6 is for matrix-vector multiplications, where broadcasting small vectors is almost
always efficient.

In [ ]:
from dist_stat.distmm import *
def test_distmm(rank, size):
    TType = torch.DoubleTensor
    p, q, r = 8, 4, 2
    if rank==0:
        fat   = TType(p, q).normal_()
        thin1 = TType(q, r).normal_()
        thin2 = TType(p, r).normal_()
    else:
        fat, thin1, thin2 = None, TType(q,r), TType(p,r)       
    # broadcast thin1 and thin2 so they have same values across all processes
    dist.broadcast(thin1,0)
    dist.broadcast(thin2,0)
    # distribute the matrices
    fat_dist   = distmat.dist_data(fat, src=0, TType=TType)
    thin1_dist = distmat.dist_data(thin1, src=0, TType=TType)
    thin2_dist = distmat.dist_data(thin2, src=0, TType=TType)
    
    if rank==0:
        print("Scenario 1: thin2^T x fat" ) 
        correct = torch.mm(torch.t(thin2), fat)
        print(correct)
    dist.barrier()
    # (r x [p]) x ([p] x q) = (r x [q]). 
    # `out_sizes` gives information on how to split [q]. Available only if r != q.
    rslt_dist = distmat.mm(thin2_dist.t(), fat_dist, out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    assert not rslt_dist.byrow
    dist.barrier()
    
    if rank==0:
        print("Scenario 1 (transposed): fat^T x thin2" )
        correct = torch.mm(torch.t(fat), thin2)
        print(correct)
    # (q x [p]) x ([p] x r) = ([q] x r). 
    # `out_sizes` gives information on how to split q.
    rslt_dist = distmat.mm(fat_dist.t(), thin2_dist, out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    assert rslt_dist.byrow
    
    if rank==0:
        print("Scenario 2: fat x thin1")
        correct = torch.mm(fat, thin1)
        print(correct)
    dist.barrier()
    # ([p] x q) x ([q] x r) = ([p] x r).
    rslt_dist = distmat.mm(fat_dist, thin1_dist) 
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    assert rslt_dist.byrow
    dist.barrier()

    if rank==0:
        print("Scenario 2 (transposed): thin1^T x fat^T")
        correct = torch.mm(torch.t(thin1), torch.t(fat))
        print(correct)
    dist.barrier()
    # (r x [q]) x (q x [p]) = (r x [p]).
    rslt_dist = distmat.mm(thin1_dist.t(), fat_dist.t())
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    assert not rslt_dist.byrow
    dist.barrier()

    if rank==0: 
        print("Scenario 3: thin1^T x thin1")
        thin1_thin1 = torch.mm(torch.t(thin1), thin1)
        print("correct: ", thin1_thin1)
    dist.barrier()
    # (r x [p]) x ([p] x s) = (r x s).
    # Selected when no `out_sizes` is given.
    thin1_thin1_bd = distmat.mm(thin1_dist.t(), thin1_dist)
    print("rslt in rank %d: "%(rank,), thin1_thin1_bd)
    dist.barrier()

    if rank==0:
        print("Scenario 4: thin1 x thin2^T" )
        correct = torch.mm(thin1, torch.t(thin2))
        print("correct: ", correct)
    dist.barrier()
    # ([p] x r) x (r x [q]) = ([p] x q).
    rslt_dist = distmat.mm(thin1_dist, thin2_dist.t())
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    assert rslt_dist.byrow

    if rank==0:
        print("Scenario 5: thin2 x thin1_thin1")
        correct = torch.mm(thin2, thin1_thin1)
        print("correct: ", correct)
    dist.barrier()
    rslt_dist = distmat.mm(thin2_dist, thin1_thin1_bd)
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    assert rslt_dist.byrow # the result must be row-major
    dist.barrier()

    if rank==0:
        print("Scenario 5 (transposed): thin1_thin1 x thin2^T")
        correct = torch.mm(thin1_thin1, torch.t(thin2))
        print("correct: ", correct)
    dist.barrier()
    # ([p] x r) x (r x s) = ([p] x s).
    rslt_dist = distmat.mm(thin1_thin1_bd, thin2_dist.t())
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    assert not rslt_dist.byrow # the result is col-major
    dist.barrier()

    if rank==0:
        print("Scenario 6: thin1^T x thin1 (local broadcasted)" )
        correct = torch.mm(torch.t(thin1), thin1)
        print("correct: ", correct)
    # (r x [p]) x (p x s) = (r x s).
    rslt_dist = distmat.mm(thin1_dist.t(), thin1) #distmm_db_b(thin1_dist.t(), thin1)
    print("rslt in rank %d (local broadcasted): "%(rank,), rslt_dist)
    dist.barrier()
 

In [ ]:
run_process(4, test_distmm)

As stated before, distributed matrix may have a sparse matrix as local data.

In [ ]:
def mm_sparse(rank, size):
    def to_sparse(x):
        """ converts dense tensor x to sparse format """
        x_typename = torch.typename(x).split('.')[-1]
        sparse_tensortype = getattr(torch.sparse, x_typename)

        indices = torch.nonzero(x)
        if len(indices.shape) == 0:  # if all elements are zeros
            return sparse_tensortype(*x.shape)
        indices = indices.t()
        values = x[tuple(indices[i] for i in range(indices.shape[0]))]
        return sparse_tensortype(indices, values, x.size())

    TType = torch.DoubleTensor
    q, r = 4, 2
    if rank==0:
        thin1 = TType(q, r).normal_()
    else:
        thin1 = TType(q,r)
        
    # broadcast thin1 and thin2 so they have same values across all processes
    dist.broadcast(thin1,0)
    
    # distribute the matrices
    thin1_dist = distmat.dist_data(thin1, src=0, TType=TType) 

    # construct sparse matrices (no actual zero in this case, but uses sparse matrix data structure)
    thin1_sparse_chunk = to_sparse(thin1_dist.chunk)
    print("Sparse x dense, Scenario 3: in rank ", rank, "we have: ", thin1_sparse_chunk)
    thin1_sparse_dist = THDistMat.from_chunks(thin1_sparse_chunk)

    if rank==0:
        print("correct: ", torch.mm(thin1.t(), thin1))

    r =  distmat.mm(thin1_sparse_dist.t(), thin1_dist )
    print("rslt: ", r)

In [ ]:
run_process(4, mm_sparse)

## Nonnegative Matrix Factorization (NMF)

Approximate a nonnegative data matrix $X \in \mathbb{R}^{m \times p}$ by $VW$, $V \in \mathbb{R}^{m \times r}$ and $W \in \mathbb{R}^{r \times p}$. In a simple setting, NMF minimizes
\begin{align*}
f(V, W) =  \|X - VW\|_\mathrm{F}^2.
\end{align*}

Multiplicative algorithm [Lee and Seung, 1999, 2001]:
\begin{align*}
V^{n+1} &= V^n \odot [X (W^n)^T] \oslash [V^n W^n (W^n)^T] \\
W^{n+1} &= W^n \odot [(V^{n+1})^T X] \oslash [(V^{n+1})^T V^{n+1} W^n],
\end{align*}
where $\odot$ and $\oslash$ denote elementwise multiplication and division.

The following code is a simplified version. The full object-oriented version is included in our package.

In [ ]:
def nmf(rank, size):
    p = 8; q = 12; r = 3
    maxiter = 1000
    TensorType=torch.DoubleTensor
    data = distmat.distgen_uniform(p, q, TType=TensorType)
    V = distmat.distgen_uniform(p, r, TType=TensorType)
    W = distmat.distgen_uniform(q, r, TType=TensorType).t()
    for i in range(maxiter):
        XWt =  distmat.mm(data, W.t())
        WWt =  distmat.mm(W, W.t())
        VWWt = distmat.mm(V, WWt)
        V.mul_(XWt).div_(VWWt)

        VtX  = distmat.mm(V.t(), data, out_sizes=W.sizes)
        VtV  = distmat.mm(V.t(), V)
        VtVW = distmat.mm(VtV, W)
        W = W.mul_(VtX).div_(VtVW)
        if (i+1) % 100 == 0:
            # print objective
            outer = distmat.mm(V, W)
            val = ((data - outer)**2).sum()
            if rank == 0:
                print("Iteration {}: {}".format(i+1, val))

In [ ]:
run_process(4, nmf)

Inside the package `dist_stat`, we have implemented...

* Nonnegative matrix factorization:
    * Multiplicative method
    * Alternating proximal gradient method
* Positron Emission Tomography:
    * with $\ell_2$-penalty, MM method
    * with $\ell_1$-penlaty, primal-dual method
* Multidimensional Scaling
* $\ell_1$-regularized Cox regression

In [ ]:
def nmf_mult(rank, size):
    import dist_stat.nmf as nmf
    p = 8; q = 12; r = 3
    maxiter = 3000
    TensorType=torch.DoubleTensor
    torch.manual_seed(100)
    data = distmat.distgen_uniform(p, q, TType=TensorType, set_from_master=True) # to guarantee same input matrices throughout experiments
    driver = nmf.NMF(data, r)
    V, W = driver.run(maxiter=maxiter, tol=1e-5, check_interval=100, check_obj=True) 
    # if check_obj=False, the objective value is not estimated.
    # the convergence is determined based on maximum change in V and W.

In [ ]:
run_process(4, nmf_mult)

Alternating projected gradient (APG) with ridge penalties:

\begin{align*}
f(V, W; \epsilon) =  \|X - VW\|_\mathrm{F}^2 + \frac{\epsilon}{2} \|V\|_\mathrm{F}^2 + \frac{\epsilon}{2} \|W\|_\mathrm{F}^2
\end{align*}
is minimized. 
The corresponding APG update is given by
\begin{align*}
V^{n+1} &= P_+ \left((1 - \sigma_n \epsilon) V^n - \sigma_n (V^n W^n (W^n)^T - X (W^n)^T) \right) \\
W^{n+1} &= P_+ \left((1 - \tau_n \epsilon) W^n - \tau_n ((V^{n+1})^T V^{n+1} W^n - (V^{n+1})^TX ) \right).
\end{align*}

The below is the APG with $\epsilon=0$.

In [ ]:
def nmf_apg(rank, size):
    import dist_stat.nmf_pg as nmf
    p = 8; q = 12; r = 3
    maxiter = 3000
    TensorType=torch.DoubleTensor
    torch.manual_seed(100)
    data = distmat.distgen_uniform(p, q, TType=TensorType, set_from_master=True)
    driver = nmf.NMF(data, r)
    V, W = driver.run(maxiter=maxiter, tol=1e-5, check_interval=100, check_obj=True)

In [ ]:
run_process(4, nmf_apg)

When it goes large-scale with GPU or SIMD acceleration, the time difference becomes much smaller with faster convergence.

## $\ell_1$-regularized Cox Regression

Log-partial likelihood of Cox proportional hazards model:
\begin{align*}
L (\beta) = \sum_{i=1}^m \delta_i \left[\beta^T x_i - \log \left(\sum_{j: y_j \ge y_i} \exp(\beta^T x_j)\right)\right]. 
\end{align*}

* $y_i = \min \{t_i, c_i\}$
    * $t_i$: time to event
    * $c_i$: right-censoring time for that sample
* $\delta = (\delta_1, \dotsc, \delta_m)^T$
    * $\delta_i= I_{\{t_i \le c_i\}}$: indicator for censoredness of sample $i$.  

\begin{align*}
\nabla L(\beta) = X^T (I-P) \delta,
\end{align*} 
  
$w_i = \exp(x_i^T \beta)$, $W_j = \sum_{i: y_i \ge y_j} w_i$, $P = (\pi_{ij})$ where
$$
\pi_{ij} = I(y_i \ge y_j) w_i/W_j.
$$

Using the proximal gradient method:

\begin{align*}
w_i^{n+1} &= \exp(x_i^T \beta); \;\; W_j^{n+1} = \sum_{i: y_i \ge y_j} w_i^{n+1}\\
\pi_{ij}^{n+1} &= I(t_i \ge t_j) w_i^{n+1} / W_j^{n+1} \\
\Delta^{n+1} &= X^T (I - P^{n+1}) \delta, \;\; \text{where $P^{n+1} = (\pi_{ij}^{n+1})$} \\
\beta^{n+1} &= \mathcal{S}_{\lambda}(\beta^n + \sigma \Delta^{n+1}),
\end{align*}

* $\mathcal{S}$ is the soft-thresholding operator:
$$
  [\mathcal{S}_{\lambda}(u)]_i = \mathrm{sign}(u_i) (|u_i| - \lambda)_+.
$$
* Convergence guaranteed when $\sigma \le 1/(2 \|X\|_2^2)$.
* $W_j$ can be computed using `cumsum` function when the data are sorted in decreasing order of $y_i$.

In [ ]:
def cox_l1(rank, size):
    import dist_stat.cox as cox
    n = 8; p = 12
    lambd=0.001
    maxiter=3000
    torch.manual_seed(100)
    TensorType = torch.DoubleTensor
    # The below is how one would create a column-major matrix.
    # We want column-major matrix to invoke matrix multiplication scenario 3. (`beta` is distributed.)
    X = distmat.distgen_normal(p, n, TType=TensorType, set_from_master=True).t()     
    torch.manual_seed(200)
    delta = torch.multinomial(torch.tensor([1., 1.]), n, replacement=True).float().view(-1, 1).type(TensorType) # 50% censored, 50% noncensored
    dist.broadcast(delta, 0) # same delta shared across processes
    cox_driver = cox.COX(X, delta, lambd, seed=300, TType=TensorType, sigma='power') # power iteration to estimate matrix norm
    beta = cox_driver.run(maxiter, tol=1e-5, check_interval=100, check_obj=True)
    zeros = (beta == 0).type(torch.int64).sum()
    if rank == 0:
        print("number of zeros:", zeros)

In [ ]:
run_process(4, cox_l1)

## Multi-GPU Demonstration

We demonstrate 10,000 x 10,000 examples on 2-8 GPUs on our server. The scripts in the `examples` directory are designed to automatically select the GPU device.

## Multi-node

The data structure can also be utilized on multi-node clusters. The structure was used for the analysis of 200,000 x 500,000 UK Biobank data.

## Future Direction

MPI-only, lightweight, more flexible version in Julia is in preparation. CUDA-aware MPI support for the central MPI interface [MPI.jl](https://github.com/JuliaParallel/MPI.jl) was added in the process.